## 1. Clone BIG Project

In [ ]:
%cd "/content/"
!git clone https://github.com/jina-ai/big_creative_ai.git
%cd "big_creative_ai"

## 2. Install Requirements

In [ ]:
%pip uninstall tensorflow -y
%pip install -r BIGDreamBooth/requirements-colab.txt
%pip install -qq git+https://github.com/ShivamShrirao/diffusers.git@0a5948e7f49ea47edeaf8a617b2e011aecfefa23
%pip install -q -U triton==2.0.0.dev20221202
%pip install --no-deps -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl

## 3. Import and Login to Hubble

In [ ]:
from torch import multiprocessing
multiprocessing.set_start_method('spawn')

import os
os.environ['JINA_LOG_LEVEL'] = 'DEBUG'

## 5. Start `BIGDreamBoothExecutor` in a Flow

In [ ]:
from jina import Flow
from BIGDreamBooth.executor import BIGDreamBoothExecutor

os.environ['JINA_LOG_LEVEL'] = 'DEBUG'

hf_token = ''

f = Flow(env={'JINA_LOG_LEVEL': 'DEBUG'}).add(
    uses=BIGDreamBoothExecutor,
    uses_with={
        'hf_token': hf_token,
        'is_colab': True
    },
    env={'JINA_LOG_LEVEL': 'DEBUG'}
  )

f.start()

## 6. Generate Images with Pretrained Stable Diffusion


In [ ]:
import time
from pathlib import Path

from docarray import DocumentArray, Document

object_style_identifier = None
prompt = f'an austronaut riding a horse on the moon'
# 'private' for using private model, 'meta' for using metamodel, 'pretrained' for using pretrained model
target_model = 'pretrained'

num_images = 10


# update prompt with category of used identifiers
identifier_n_categories = f.post(
    on='/list_identifiers_n_categories',
)
identifier_n_categories = identifier_n_categories[0].tags.get(target_model, {})
for _identifier, _category in identifier_n_categories.items():
    prompt = prompt.replace(_identifier, f"{_identifier} {_category}")

# generate images
folder_images_prefix = '../generated_images'
if target_model == 'private':
    folder_images_prefix += f'/{object_style_identifier}'
elif target_model == 'meta':
    folder_images_prefix += f'/metamodel'
elif target_model == 'pretrained':
    folder_images_prefix += f'/pretrained'
elif target_model == 'private_meta':
    folder_images_prefix += f'/private_metamodel'
else:
    raise ValueError(f"Unknown target_model '{target_model}'")
folder_images = Path(f"{folder_images_prefix}/{prompt.replace(' ', '-').replace(',', '')}")
folder_images = Path(f"{str(folder_images)[:200]}-{time.time()}")

image_docs: DocumentArray = f.post(
    on='/generate',
    inputs=Document(text=prompt),
    parameters={
        'identifier': object_style_identifier,
        'target_model': target_model,
        'num_images': num_images,
    }
)
folder_images.mkdir(exist_ok=True, parents=True)
for i, image_doc in enumerate(image_docs[0].chunks):
    image_doc.save_blob_to_file(f"{str(folder_images)}/generation-{i}.png")

print(f"Generations were successful and were saved to {folder_images}")



## Fine-tune Stable Diffusion To Your Images

In [ ]:
from docarray import DocumentArray


# specify the path to the images
path_to_instance_images = '../path_to_images'
# specify the category of the images, this could be e.g. 'painting', 'dog', 'bottle', etc.
category = '[category of images]'
# 'private' for training private model from pretrained model, 'meta' for training metamodel
target_model = 'private'

# some custom parameters for the training
max_train_steps = 200
learning_rate = 1e-6


docs = DocumentArray.from_files(f'{path_to_instance_images}/**')
for doc in docs:
    doc.load_uri_to_blob()
    doc.uri = None


identifier_doc = f.post(
    on='/finetune',
    inputs=docs,
    parameters={
        'category': category,
        'target_model': target_model,
        'learning_rate': learning_rate,
        'max_train_steps': max_train_steps,
    },
)

print(f"Finetuning was successful. The identifier for the object is '{identifier_doc[0].text}'")

## 7. Generate Images for Fine-tuned Model

In [ ]:
import time
from pathlib import Path

from docarray import DocumentArray
from jina import Document

object_style_identifier = 'sks'
prompt = f'a {object_style_identifier}'
# 'private' for using private model, 'meta' for using metamodel, 'pretrained' for using pretrained model
target_model = 'private'

num_images = 10


# update prompt with category of used identifiers
identifier_n_categories = f.post(
    on='/list_identifiers_n_categories',
)
print(identifier_n_categories[0].tags)
identifier_n_categories = identifier_n_categories[0].tags.get(target_model, {})
for _identifier, _category in identifier_n_categories.items():
    prompt = prompt.replace(_identifier, f"{_identifier} {_category}")

# generate images
folder_images_prefix = '../generated_images'
if target_model == 'private':
    folder_images_prefix += f'/{object_style_identifier}'
elif target_model == 'meta':
    folder_images_prefix += f'/metamodel'
elif target_model == 'pretrained':
    folder_images_prefix += f'/pretrained'
elif target_model == 'private_meta':
    folder_images_prefix += f'/private_metamodel'
else:
    raise ValueError(f"Unknown target_model '{target_model}'")
folder_images = Path(f"{folder_images_prefix}/{prompt.replace(' ', '-').replace(',', '')}")
folder_images = Path(f"{str(folder_images)[:200]}-{time.time()}")

image_docs: DocumentArray = f.post(
    on='/generate',
    inputs=Document(text=prompt),
    parameters={
        'identifier': object_style_identifier,
        'target_model': target_model,
        'num_images': num_images,
    }
)
folder_images.mkdir(exist_ok=True, parents=True)
for i, image_doc in enumerate(image_docs[0].chunks):
    image_doc.save_blob_to_file(f"{str(folder_images)}/generation-{i}.png")

print(f"Generations were successful and were saved to {folder_images}")



## 8. Repeat

Now you can fine-tune more `private`, public `meta` models or `private_meta` models and generate images with thems.